In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from tensorflow import keras
import tensorflow as tf
from skimage.transform import resize
from scipy.ndimage import gaussian_filter
from skimage import data,io,img_as_float
from skimage.morphology import reconstruction
from sklearn.model_selection import train_test_split

In [ ]:
def read_data():
    raw_data_path = os.path.join(os.path.pardir,'data','raw')
    train_data_path = os.path.join(raw_data_path,'train.csv')
    depth_data_path = os.path.join(raw_data_path,'depths.csv')
    df_train = pd.read_csv(train_data_path,index_col='id')
    df_train.rle_mask.fillna(-1,inplace=True)
    df_depth = pd.read_csv(depth_data_path,index_col='id')
    df_train['depth'] = df_depth['z']
    return df_train

In [ ]:
df = read_data()

In [ ]:
plt.rcParams["figure.figsize"] = (5,10)
plt.hist(df['depth'].values,bins=20)

In [ ]:
ids=['000e218f21','0b73b427d1','0ba541766e','0a7e067255','0a1742c740']
raw_data_path = os.path.join(os.path.pardir,'data','raw')
train_images_path = os.path.join(raw_data_path,'train','images')
train_masks_path = os.path.join(raw_data_path,'train','masks')
plt.rcParams["figure.figsize"] = (40,3)
for i,imgname in enumerate(ids):  
    img = mpimg.imread(train_images_path + '\\' + imgname + '.png')
    img_mask = mpimg.imread(train_masks_path + '\\' + imgname + '.png')
    plt.subplot(1,(len(ids)+1)*2,(i+1)*2-1)
    plt.imshow(img)
    plt.subplot(1,(len(ids)+1)*2,(i+1)*2)
    plt.imshow(img_mask)
plt.show()

In [ ]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
# resize image & convert into numpy

def resizeimage(img,targetSize):
    return resize(img, (targetSize, targetSize), mode='constant', preserve_range=True)
    

In [ ]:
# read image file and convert into numpy array
def convertImage2Array(imgname,rawpath):
    image = io.imread(rawpath + '\\' + imgname + '.png',as_gray=True)
    #resize and normalize data
    resizedImage = resizeimage(image,128)
    return np.array(img_as_float(resizedImage))

train_images_path = os.path.join(raw_data_path,'train','images')
df['image'] = [convertImage2Array(imgname,train_images_path) for imgname in df.index.values]

train_masks_path = os.path.join(raw_data_path,'train','masks')
df['mask_img'] = [convertImage2Array(imgname,train_masks_path) for imgname in df.index.values]

In [ ]:
def build_model(input_layer, start_neurons):
    # 128 -> 64
    conv1 = tf.keras.layers.Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = tf.keras.layers.Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = tf.keras.layers.MaxPooling2D((2, 2))(conv1)
    pool1 = tf.keras.layers.Dropout(0.25)(pool1)

    # 64 -> 32
    conv2 = tf.keras.layers.Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = tf.keras.layers.Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = tf.keras.layers.MaxPooling2D((2, 2))(conv2)
    pool2 = tf.keras.layers.Dropout(0.5)(pool2)

    # 32 -> 16
    conv3 = tf.keras.layers.Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)
    conv3 = tf.keras.layers.Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(conv3)
    pool3 = tf.keras.layers.MaxPooling2D((2, 2))(conv3)
    pool3 = tf.keras.layers.Dropout(0.5)(pool3)

    # 16 -> 8
    conv4 = tf.keras.layers.Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(pool3)
    conv4 = tf.keras.layers.Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(conv4)
    pool4 = tf.keras.layers.MaxPooling2D((2, 2))(conv4)
    pool4 = tf.keras.layers.Dropout(0.5)(pool4)

    # Middle
    convm = tf.keras.layers.Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(pool4)
    convm = tf.keras.layers.Conv2D(start_neurons * 16, (3, 3), activation="relu", padding="same")(convm)

    # 8 -> 16
    deconv4 = tf.keras.layers.Conv2DTranspose(start_neurons * 8, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv4 = tf.keras.layers.concatenate([deconv4, conv4])
    uconv4 = tf.keras.layers.Dropout(0.5)(uconv4)
    uconv4 = tf.keras.layers.Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)
    uconv4 = tf.keras.layers.Conv2D(start_neurons * 8, (3, 3), activation="relu", padding="same")(uconv4)

    # 16 -> 32
    deconv3 = tf.keras.layers.Conv2DTranspose(start_neurons * 4, (3, 3), strides=(2, 2), padding="same")(uconv4)
    uconv3 = tf.keras.layers.concatenate([deconv3, conv3])
    uconv3 = tf.keras.layers.Dropout(0.5)(uconv3)
    uconv3 = tf.keras.layers.Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)
    uconv3 = tf.keras.layers.Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(uconv3)

    # 32 -> 64
    deconv2 = tf.keras.layers.Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(uconv3)
    uconv2 = tf.keras.layers.concatenate([deconv2, conv2])
    uconv2 = tf.keras.layers.Dropout(0.5)(uconv2)
    uconv2 = tf.keras.layers.Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = tf.keras.layers.Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)

    # 64 -> 128
    deconv1 = tf.keras.layers.Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = tf.keras.layers.concatenate([deconv1, conv1])
    uconv1 = tf.keras.layers.Dropout(0.5)(uconv1)
    uconv1 = tf.keras.layers.Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = tf.keras.layers.Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)

    uncov1 = tf.keras.layers.Dropout(0.5)(uconv1)
    output_layer = tf.keras.layers.Conv2D(1, (1,1), padding="same", activation="sigmoid")(uconv1)
    
    return output_layer


img_size_target = 128
input_layer = tf.keras.Input((img_size_target, img_size_target, 1))
output_layer = build_model(input_layer, 16)

In [ ]:
model = tf.keras.Model(input_layer,output_layer)
model.compile(optimizer='adam',metrics=['accuracy'],loss='binary_crossentropy')

In [ ]:
model.summary()

In [ ]:
#train & test splitt

ids_train,ids_valid,X_train,X_valid,y_train,y_valid,depth_train,depth_valid = train_test_split(
    df.index.values.tolist(),
    np.array(df.image.values.tolist()).reshape(-1,128,128,1),
    np.array(df.mask_img.values.tolist()).reshape(-1,128,128,1),
    df.depth.values,
    test_size=0.2
    )

X_train = np.append(X_train, [np.fliplr(x) for x in X_train], axis=0)
y_train = np.append(y_train, [np.fliplr(x) for x in y_train], axis=0)

In [ ]:

early_stopping = tf.keras.callbacks.EarlyStopping(patience=10,verbose=1)
check_point = tf.keras.callbacks.ModelCheckpoint('./keras.model', save_best_only=True)
board = keras.callbacks.TensorBoard(log_dir='./logs')
#Reduce learning rate when a metric has stopped improving.
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)
epochs = 200
batch_size = 32
history = model.fit(X_train,y_train,validation_data=[X_valid,y_valid],batch_size=batch_size,epochs=epochs,callbacks=[early_stopping,check_point,board,reduce_lr])

In [ ]:
X_train[0]